In [1]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import spacy
from collections import Counter

importlib.reload(funcs)

<module 'funcs' from 'D:\\_projekty\\praca\\funcs.py'>

In [2]:
sp = pd.read_csv('00_dane/rawest.csv',  sep='|', encoding='UTF-8')

In [3]:
len(sp)

158882

In [4]:
sp.head(2)

,date_,speech_rawer,full_name,party_final,speech_rawest
0,01-02-1992,Chciałbym kilka zdań powiedzieć na temat skut...,Aleksander Bentkowski,Polskie Stronnictwo Ludowe,chcieć zdanie powiedzieć temat skutek ustawa z...
1,01-02-1992,"Przysłuchując się tej bardzo emocjonalnej, dł...",Aleksander Kwaśniewski,Sojusz Lewicy Demokratycznej,przysłuchiwać emocjonalny długo trwać debata o...


In [5]:
%%time
sp['speech_rawest'] = sp['speech_rawest'].apply(lambda x: re.sub('[\W\d]',' ', x.lower()))

#czas dla apply : 19s

AttributeError: 'float' object has no attribute 'lower'

In [17]:
nlp = spacy.load('pl_model')
stop = set(nlp.Defaults.stop_words)
stop.add('zł')
stop.add('pkt')
stop.add('art')
stop.add('ustawy')
stop.add('r')

In [18]:
%%time
sp['speech_rawest'] = sp['speech_rawest'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))

Wall time: 7.49 s


### Ile mamy słów

In [19]:
%%time
corp_dict={}
n = 1
for speech in sp['speech_rawest']:
    if n % 10000 == 0:
        print('Wystąpienie nr: ', n)
    for word in speech.split():
        if word not in corp_dict.keys(): 
            corp_dict[word]=1
        else:
            corp_dict[word]+=1
    n+=1

Wystąpienie nr:  10000
Wystąpienie nr:  20000
Wystąpienie nr:  30000
Wystąpienie nr:  40000
Wystąpienie nr:  50000
Wystąpienie nr:  60000
Wystąpienie nr:  70000
Wystąpienie nr:  80000
Wystąpienie nr:  90000
Wystąpienie nr:  100000
Wystąpienie nr:  110000
Wystąpienie nr:  120000
Wystąpienie nr:  130000
Wystąpienie nr:  140000
Wystąpienie nr:  150000
Wall time: 12.2 s


In [21]:
len(corp_dict.keys()), sum(corp_dict.values())

(152755, 42509313)

#### Sortowanie słownika po liczbie wystąpień

In [22]:
corp_dict = {k: v for k, v in sorted(corp_dict.items(), key=lambda item: item[1])}

In [23]:
corp_dict['suweren']

542

#### Jaki jest rozkład częstości słów?

Jakie słowa pojawiają się najrzadziej?

In [24]:
u = [k for k,v in corp_dict.items() if v<=5]

In [25]:
len(u)

101747

In [29]:
for _ in range(10):
    print(u[random.randrange(len(u))])

mikotoksyn
rznakowy
pompeo
własciwość
gwark
siemiński
przeładunkowość
produkty
normandyjski
szumowa


Jakie słowa pojawiają się najczęściej?

In [30]:
sorted(corp_dict.items(), key=lambda item: item[1])[-10:]

[('prawo', 221153),
 ('komisja', 227835),
 ('minister', 236987),
 ('móc', 241134),
 ('polski', 245334),
 ('państwo', 261503),
 ('projekt', 310992),
 ('rok', 392826),
 ('mieć', 521144),
 ('ustawa', 531702)]

#### Ile słów jest w każdym wystąpieniu?

In [31]:
sp.to_csv('00_dane/rawest.csv', sep='|', encoding='UTF-8', index=False)
